### Что такое проблема многорукого бандита? 
Это классический случай, который касается различных областей, от маркетинга до медицинских испытаний: как вы даете пользователю то, что он хочет, до того, как вы начнете с ним отношения? Как насчет случая, когда вы даже не знаете многого о сродстве пользователя к предмету или лечению, которое вы даете пользователю, как в случае медицинского испытания нескольких лекарств или добавок?

Посмотрим, как решать эти проблемы, и как многорукий бандит с сэмплированием Томпсона, как правило, является лучшим выбором. 

В качестве примера представим, что проводится маркетинговая кампания для  веб-сайта и у есть два объявления на выбор. И скажем, цель состоит в том, чтобы показать объявление с самым высоким значением рейтинга кликов (CTR), чтобы привлечь максимально возможный трафик. Но у нас нет предварительной информации о том, как будет работать любое из этих объявлений. Каким будет наш подход? Как насчет типичного A/B-тестирования? Как насчет того, чтобы показывать оба объявления одинаково, а затем в какой-то момент переключиться на объявление с самым высоким измеренным CTR? Как долго нужно показывать оба объявления, прежде чем остановиться на «победителе»? В этих случаях кажется, что угадывание может быть  лучшим выбором. На самом деле нет.

Есть метод для этой  проблемы называется многорукий бандит (MAB) .

По сути, алгоритм MAB является почти оптимальным методом решения дилеммы компромисса между поиском и использованием эксплойта, которая возникает, когда мы не знаем, следует ли исследовать возможные варианты, чтобы найти вариант с наилучшей отдачей, или использовать вариант, который, по нашему мнению, является наиболее выгодным. 

Рассмотрим различные алгоритмы для решения проблемы MAB. Все они имеют разные подходы и разные соотношения разведки и эксплуатации.

Определим CTR как отношение количества кликов по объявлению к количеству показов. Например, если объявление было показано 100 раз и на него кликнули 10 раз, CTR = 10/100 = 0,1.

Также определим regret как разницу между максимально возможным CTR и показанным CTR. Например, если объявление А имеет известный CTR 0,1, а объявление Б имеет известный CTR 0,3, каждый раз, когда мы показываем объявление А, мы получаем regret, равное 0,3 - 0,1 = 0,2. Это кажется небольшой разницей, пока не учтем, что реклама может быть показана 1 миллион раз всего за несколько часов.

Цель проекта понять, какая реализация алгоритма работает лучше всего с точки зрения минимизации regret. Четыре реализации, которые будем использовать:

Случайный выбор
Эпсилон Жадный
Томпсон Сэмплинг
Верхняя доверительная граница (UCB1)
Каждый метод будет описан ниже в моделировании.

Чтобы провести этот эксперимент, нужно предположить, что CTR известен заранее. Таким образом, мы можем имитировать клик данного объявления. Например, если мы показываем объявление А с известным CTR 28%, предположим, что объявление будет нажиматься в 28% случаев, и используем в симуляции.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import beta, bernoulli
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
import random
import math

RANDOM_SEED = 123
np.random.seed(RANDOM_SEED)
init_notebook_mode(connected=True)

In [53]:
def algorithm_performance():
    """
    Функция, которая покажет производительность каждого алгоритма, который мы будем использовать.
    """
    
    ## подсчитываем, сколько раз было выбрано каждое объявление
    count_series = pd.Series(index_list).value_counts(normalize=True)
    print('Объявление №0 было показано', count_series[0]*100, '% всех случаев .')
    print('Объявление №1 было показано', count_series[1]*100, '% всех случаев.')
    
    print('Общая награда (количество кликов):', total_reward) 
    
    x = np.arange (0, n, 1)

    ## график рассчитанного CTR для объявления № 0
    data1 = go.Scatter(x=x,
                       y=ctr[0],
                       name='Расчетный CTR №0',
                       line=dict(color=('rgba(10, 108, 94, .7)'),
                                 width=2))

    ##  линия с фактическим CTR для объявления № 0
    data2 = go.Scatter(x=[0, n],
                       y=[ACTUAL_CTR[0]] * 2,
                       name='Фактическое значение CTR №0',
                       line = dict(color = ('rgb(205, 12, 24)'),
                                   width = 1,
                                   dash = 'dash'))

    ## график рассчитанного CTR для объявления №1
    data3 = go.Scatter(x=x,
                       y=ctr[1],
                       name='Расчетный CTR №1',
                       line=dict(color=('rgba(187, 121, 24, .7)'),
                                 width=2))

    ## линия с фактическим CTR для объявления № 0
    data4 = go.Scatter(x=[0, n],
                       y=[ACTUAL_CTR[1]] * 2,
                       name='Фактическое значение CTR №1',
                       line = dict(color = ('rgb(205, 12, 24)'),
                                   width = 1,
                                   dash = 'dash'))

    ## значене regret как функция номера испытания
    data5 = go.Scatter(x=x,
                       y=regret_list,
                       name='Regret')

    layout = go.Layout(title='Смоделированные значения CTR и алгоритм Regret',
                       xaxis={'title': 'Колличество попыток'},
                       yaxis1={'title': 'CTR'},
                       yaxis2={'title': 'Regret'}
                       )
    fig = tools.make_subplots(rows=2, cols=1, print_grid=False, shared_yaxes=True, shared_xaxes=True)

    fig.append_trace(data1, 1, 1)
    fig.append_trace(data2, 1, 1)
    fig.append_trace(data3, 1, 1)
    fig.append_trace(data4, 1, 1)
    fig.append_trace(data5, 2, 1)

    fig['layout'].update(layout)
    iplot(fig, show_link=False)

Это не происходит в реальной жизни, но для эксперимента мы предположим, что мы знаем фактические значения CTR для обоих объявлений для целей моделирования. 

In [54]:
ACTUAL_CTR = [.45, .65]
print('Фактический CTR для объявления №0:', ACTUAL_CTR[0])
print('Фактический CTR для объявления №1:', ACTUAL_CTR[1])

Фактический CTR для объявления №0: 0.45
Фактический CTR для объявления №1: 0.65


# Случайный выбор 
0% - Разведка 

100% - Эксплуатация

Начнем с самого наивного подхода. Алгоритм случайного выбора не выполняет никаких исследований, он просто случайным образом выбирает объявление для показа. если у нас есть 2 объявления, каждое из них будет появляться примерно в 50% (= 100%/2) случаев. 

In [55]:
## Для каждого алгоритма мы проведем 1000 испытаний
n = 1000

In [56]:
regret = 0 
total_reward = 0
regret_list = [] ## писок для сбора значений сожалений для каждого показа (пробная версия)
ctr = {0: [], 1: []} ## списки для сбора рассчитанного CTR 
index_list = [] ## список для сбора количества случайно выбранных объявлений

## начальные значения для показов и кликов 
impressions = [0,0] 
clicks = [0,0]

for i in range(n):    
    
    random_index = np.random.randint(0,2,1)[0] ## случайным образом выбираем значение между [0,1]
    index_list.append(random_index) ## добавляем значение в список
    
    impressions[random_index] += 1 ## добаляем 1 значение показа для выбранного объявления
    did_click = bernoulli.rvs(ACTUAL_CTR[random_index]) ## имитируем, если человек нажал на объявление, используя фактическое значение CTR
    
    if did_click:
        clicks[random_index] += did_click ## если человек нажал, добавляет 1 значение клика для выбранного объявления
    
    ## рассчитывает значения CTR и добавляет их в список
    if impressions[0] == 0:
        ctr_0 = 0
    else:
        ctr_0 = clicks[0]/impressions[0]
        
    if impressions[1] == 0:
        ctr_1 = 0
    else:
        ctr_1 = clicks[1]/impressions[1]
        
    ctr[0].append(ctr_0)
    ctr[1].append(ctr_1)
    
    ## рассчитывает regret и вознаграждение
    regret += max(ACTUAL_CTR) - ACTUAL_CTR[random_index]
    regret_list.append(regret)
    total_reward += did_click

In [57]:
algorithm_performance()

Объявление №0 было показано 50.2 % всех случаев .
Объявление №1 было показано 49.8 % всех случаев.
Общая награда (количество кликов): 540


Оба объявления были показаны одинаковое количество раз, и чем больше испытаний, тем ближе значения CTR к их известным значениям. Однако Regret постоянно увеличивается, поскольку алгоритм ничего не изучает и не делает никаких обновлений в соответствии с полученной информацией. Это постоянно растущий Regret — именно то, что нужно свести к минимуму с помощью «умных» методов. Я бы использовал этот алгоритм в двух случаях: 1. Нужно быть уверенным в расчетном значении CTR для каждого объявления (чем больше показов получает каждое объявление, тем больше я уверен, что расчетный CTR равен реальному CTR).

In [58]:
## сохраняем значения награды и Regret  для будущего сравнения
random_dict = {'reward':total_reward, 
               'regret_list':regret_list, 
               'ads_count':pd.Series(index_list).value_counts(normalize=True)}

# Эпсилон Жадный 
~15% - Разведка

~85% - Эксплуатация 

Следующим подходом является алгоритм Эпсилон-Жадный. Его логику можно определить следующим образом: Запускаем эксперимент некоторое начальное количество раз (исследование). Выбираем выигрышный вариант с наибольшим количеством очков за начальный период исследования. 


In [59]:
e = .05 ## устанавливаем значение эпсилон
n_init = 100 ## количество показов для выбора объявления-победителя
impressions = [0,0]
clicks = [0,0]

for i in range(n_init):
    random_index = np.random.randint(0,2,1)[0]
    
    impressions[random_index] += 1
    did_click = bernoulli.rvs(ACTUAL_CTR[random_index])
    if did_click:
        clicks[random_index] += did_click
        
ctr_0 = clicks[0] / impressions[0]
ctr_1 = clicks[1] / impressions[1]
win_index = np.argmax([ctr_0, ctr_1]) ## выбираем номер объявления с самым высоким CTR

print('После', n_init, 'испытаний Объявление №', win_index, 'получило самый высокий CTR =', round(np.max([ctr_0, ctr_1]), 2), 
      '(Реальное значение CTR равно', ACTUAL_CTR[win_index], ').'
      '\nОно будет показано', (1-e)*100, '% от всего времени размещения.')

После 100 испытаний Объявление № 1 получило самый высокий CTR = 0.65 (Реальное значение CTR равно 0.65 ).
Оно будет показано 95.0 % от всего времени размещения.


In [60]:
regret = 0 
total_reward = 0
regret_list = [] 
ctr = {0: [], 1: []}
index_list = [] 
impressions = [0,0] 
clicks = [0,0]

for i in range(n):    
    
    epsilon_index = random.choices([win_index, 1-win_index], [1-e, e])[0]
    index_list.append(epsilon_index)
    
    impressions[epsilon_index] += 1
    did_click = bernoulli.rvs(ACTUAL_CTR[epsilon_index])
    if did_click:
        clicks[epsilon_index] += did_click
    
    if impressions[0] == 0:
        ctr_0 = 0
    else:
        ctr_0 = clicks[0]/impressions[0]
        
    if impressions[1] == 0:
        ctr_1 = 0
    else:
        ctr_1 = clicks[1]/impressions[1]
        
    ctr[0].append(ctr_0)
    ctr[1].append(ctr_1)
    
    regret += max(ACTUAL_CTR) - ACTUAL_CTR[epsilon_index]
    regret_list.append(regret)
    total_reward += did_click

In [61]:
algorithm_performance()

Объявление №0 было показано 4.9 % всех случаев .
Объявление №1 было показано 95.1 % всех случаев.
Общая награда (количество кликов): 610


Это намного лучше. Стоит обратить внимание, как Regret уменьшился на порядок! Алгоритм Epsilon-Greedy работает намного лучше, чем Random Selection. Но есть проблема, выигрышный вариант периода исследования не обязательно будет оптимальным вариантом, и возможно будет  использоваться неоптимальный вариант. Это увеличивает Regret и уменьшает награду. Согласно **Закону больших чисел**\*, чем больше первоначальных попыток вы сделаете, тем больше шансов, что вы правильно выберете выигрышный вариант. Но в маркетинге обычно не хочется полагаться на случай.

> \*В теории вероятностей закон больших чисел (ЗБЧ) — это теорема, описывающая результат выполнения одного и того же эксперимента большое количество раз. Согласно закону, среднее значение результатов, полученных в результате большого количества испытаний, должно быть близко к ожидаемому значению и будет становиться ближе по мере проведения большего количества испытаний..

Преимущество подхода заключается в том, что можно настроить коэффициент, определяющий частоту показа победившей рекламы после первоначальных испытаний, выбрав разные значения $\epsilon$ (эпсилон). 

In [62]:
epsilon_dict = {'reward':total_reward, 
                'regret_list':regret_list, 
                'ads_count':pd.Series(index_list).value_counts(normalize=True)}

## Томпсон Сэмплинг 
50% - Разведка 

50% - Эксплуатация 

Исследовательская часть Thompson Sampling является более сложной, чем алгоритм e-Greedy. Здесь использовалось бета-распределение, однако выборка Томпсона может быть обобщена для выборки из любых распределений по параметрам.

> \*В теории вероятностей и статистике бета-распределение — это семейство непрерывных распределений вероятностей, определенных на интервале [0, 1] параметризованных двумя положительными параметрами формы, обозначаемыми $\alpha$ and $\beta$, которые появляются как показатели случайной величины и контролируют форму распределения. 


Логика:

1. Выбираем распределения для параметров $\alpha$ и $\beta$.
2. Рассчитываем значения $\alpha$ и $\beta$ следующим образом: $\alpha = prior + hits$, $\beta = prior + misses$. * В нашем случае hits = количество кликов, misses = количество показов без клика. Priors полезны, если у есть предварительная информация о фактическом CTR объявлений. Здесь нет, поэтому будем использовать 1.0.*
3. Оцениваем фактические CTR путем выборки значений бета-распределения для каждого варианта $B(\alpha_i, \beta_i)$ и выбираем выборку с наибольшим значением (оценочный CTR).
4. Повторяем 2-3.



In [63]:
regret = 0 
total_reward = 0
regret_list = [] 
ctr = {0: [], 1: []}
index_list = [] 
impressions = [0,0] 
clicks = [0,0]
priors = (1, 1)
win_index = np.random.randint(0,2,1)[0] ## случайным образом выбрает первое показанное объявление

for i in range(n):    
    
    impressions[win_index] += 1
    did_click = bernoulli.rvs(ACTUAL_CTR[win_index])
    if did_click:
        clicks[win_index] += did_click
    
    ctr_0 = random.betavariate(priors[0]+clicks[0], priors[1] + impressions[0] - clicks[0])
    ctr_1 = random.betavariate(priors[0]+clicks[1], priors[1] + impressions[1] - clicks[1])
    win_index = np.argmax([ctr_0, ctr_1])
    index_list.append(win_index)
    
    ctr[0].append(ctr_0)
    ctr[1].append(ctr_1)
    
    regret += max(ACTUAL_CTR) - ACTUAL_CTR[win_index]
    regret_list.append(regret)    
    total_reward += did_click

In [64]:
## Строим бета-распределение
x = np.arange (0, 1, 0.01)
y = beta.pdf(x, priors[0]+clicks[0], priors[1] + impressions[0] - clicks[0])
y /= y.max() ## Нормализуем

data1 = go.Scatter(x=x,
                   y=y,
                   name='Бета-распределение (объявления №0)',
                   marker = dict(color=('rgba(10, 108, 94, 1)')),
                   fill='tozeroy',
                   fillcolor = 'rgba(10, 108, 94, .7)')

data2 = go.Scatter(x = [ACTUAL_CTR[0]] * 2,
                   y = [0, 1],
                   name = 'Фактическое значение CTR #0',
                   mode='lines',
                   line = dict(
                       color = ('rgb(205, 12, 24)'),
                       width = 2,
                       dash = 'dash'))

y = beta.pdf(x, priors[0]+clicks[1], priors[1] + impressions[1] - clicks[1])
y /= y.max()

data3 = go.Scatter(x=x,
                   y=y,
                   name='Бета-распределение (объявления №1)',
                   marker = dict(color=('rgba(187, 121, 24, 1)')),
                   fill='tozeroy',
                   fillcolor = 'rgba(187, 121, 24, .7)')

data4 = go.Scatter(x = [ACTUAL_CTR[1]] * 2,
                   y = [0, 1],
                   name = 'Фактическое значение CTR №1',
                   mode='lines',
                   line = dict(
                       color = ('rgb(205, 12, 24)'),
                       width = 2,
                       dash = 'dash'))

layout = go.Layout(title='Бета-распределения для обоих объявлений',
                   xaxis={'title': 'Возможные значения CTR'},
                   yaxis={'title': 'Плотность вероятности'})

fig = go.Figure(data=[data1, data2, data3, data4], layout=layout)



iplot(fig, show_link=False)

Общаем внимание на зону пересечения. Могут быть случаи, когда значение бета-распределения для объявления № 0 будет выше, чем для объявления № 1, поэтому алгоритм выберет объявление № 0 (которое работает хуже).

In [65]:
algorithm_performance()

Объявление №0 было показано 3.0 % всех случаев .
Объявление №1 было показано 97.0 % всех случаев.
Общая награда (количество кликов): 638


regret - самое низкий, что был до сих пор. Каждый всплеск regret происходил, когда было выбрано объявление № 0. На графике значения CTR видно, что в начале зеленые значения (значения CTR по Томпсону для объявления № 0) часто превышали светло-коричневые (значения CTR для объявления № 1 по Томпсону), в результате чего объявление № 0 показывалось.

Есть различия в вариантах для каждого объявления. Алгоритм постоянно исследует, а затем использует вариант рекламы с самой высокой выборкой, взятой из соответствующего бета-распределения. Это можно увидеть на верхнем графике распределения. Бета-распределение для объявления № 1  выше и уже, чем у объявления № 0,это означает, что его выборочные значения будут постоянно выше, чем у объявления № 0, так и должно быть.

In [66]:
thompson_dict = {'reward':total_reward, 
                 'regret_list':regret_list, 
                 'ads_count':pd.Series(index_list).value_counts(normalize=True)}

## Верхняя доверительная граница (UCB1) 
50% - Разведка

50% - Эксплуатация 

В отличие от алгоритма выборки Томпсона, верхняя доверительная граница больше заботится о неопределенности (высокой вариации) каждого варианта. Чем больше неуверенность в вариантах, тем важнее их исследовать. Алгоритм выбирает вариант с наивысшим значением верхней границы достоверности (UCB), который представляет собой максимальное предположение вознаграждения для варианта. Он определяется следующим образом:

$UCB = \bar x_i + \sqrt{\frac{2 \cdot \log{t}}{n}}$ ,

где $\bar x_i$ -  (показатель CTR) для $i$-го шага,

$t$ - общее количество (показов) по всем вариантам,

$n$ - общее количество (показов) для выбранного варианта

Логика:

1. Рассчитываем UCB для всех вариантов.
2. Выбираем вариант с наибольшим значением UCB.
3. Повторяем.



In [67]:
regret = 0 
total_reward = 0
regret_list = [] 
index_list = [] 
impressions = [0,0] 
clicks = [0,0]
ctr = {0: [], 1: []}
total_reward = 0

for i in range(n):
    
    index = 0
    max_upper_bound = 0
    for k in [0,1]:
        if (impressions[k] > 0):
            CTR = clicks[k] / impressions[k]
            delta = math.sqrt(2 * math.log(i+1) / impressions[k])
            upper_bound = CTR + delta
            ctr[k].append(CTR)
        else:
            upper_bound = 1e400
        if upper_bound > max_upper_bound:
            max_upper_bound = upper_bound
            index = k
    index_list.append(index)
    impressions[index] += 1
    reward = bernoulli.rvs(ACTUAL_CTR[index])
    
    clicks[index] += reward
    total_reward += reward
    
    regret += max(ACTUAL_CTR) - ACTUAL_CTR[index]
    regret_list.append(regret)

In [68]:
algorithm_performance()

Объявление №0 было показано 17.4 % всех случаев .
Объявление №1 было показано 82.6 % всех случаев.
Общая награда (количество кликов): 597


Видно, что Regret увеличился, когда алгоритм попытался уменьшить неопределенность CTR для объявления № 0, выбрав его. Это может быть полезно, когда вы хотите, чтобы модель в будущем чаще выбирала лучший вариант, и не заинтересованы в сильном уменьшении разведки.

In [69]:
ucb1_dict = {'reward':total_reward, 
             'regret_list':regret_list, 
             'ads_count':pd.Series(index_list).value_counts(normalize=True)}

## Сравнение и выводы 
Время сравнить четыре эти метода и посмотреть, какой из них лучше подходит для нашей задачи. Во-первых, очевидно, что нужно чаще показывать объявление №1, так как его фактический CTR равен 0,65. Посмотрим на соотношение, сколько раз правильное объявление было выбрано для каждого алгоритма.

In [71]:
data1 = go.Bar(x=['Random Selection', 'Epsilon Greedy', 'Thompson Sampling', 'UCB1'],
               y=[random_dict['ads_count'][0], 
                  epsilon_dict['ads_count'][0], 
                  thompson_dict['ads_count'][0],
                  ucb1_dict['ads_count'][0]],
               name='Ad #0',
               marker=dict(color='rgba(10, 108, 94, .7)'))

data2 = go.Bar(x=['Random Selection', 'Epsilon Greedy', 'Thompson Sampling', 'UCB1'],
               y=[random_dict['ads_count'][1], 
                  epsilon_dict['ads_count'][1], 
                  thompson_dict['ads_count'][1],
                  ucb1_dict['ads_count'][1]],
               name='Ad #1',
               marker=dict(color='rgba(187, 121, 24, .7)'))

data = [data1, data2]
layout = go.Layout(title='Соотношение появления обоих объявлений на протяжении испытаний',
                   xaxis={'title': 'Алгоритм'},
                   yaxis={'title': 'Соотношение'},
                   barmode='stack')

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Победителями здесь стали Thompson Sampling и Epsilon-Greedy, поскольку они чаще всего показывали правильное объявление №1.

In [74]:
data1 = go.Scatter(
    x=np.arange (0, n, 1),
    y=random_dict['regret_list'],
    name='Random Selection',
    marker=dict(color='#ffcc66')
)
data2 = go.Scatter(
    x=np.arange (0, n, 1),
    y=epsilon_dict['regret_list'],
    name='e-Greedy',
    marker=dict(color='#0099ff')
)
data3 = go.Scatter(
    x=np.arange (0, n, 1),
    y=thompson_dict['regret_list'],
    name='Thompson Sampling',
    marker=dict(color='#ff3300')
)
data4 = go.Scatter(
    x=np.arange (0, n, 1),
    y=ucb1_dict['regret_list'],
    name='UCB1',
    marker=dict(color='#33cc33')
)

layout = go.Layout(
    title='Regret по алгоритму',
    xaxis={'title': 'Trial'},
    yaxis={'title': 'Regret'}
)

data = [data1, data2, data3, data4]
fig = go.Figure(data=data, layout=layout)
iplot(fig)

Принимая во внимание, что алгоритмы Thompson Sampling и Epsilon-Greedy большую часть времени выбирали рекламу с более высоким CTR (#1), неудивительно, что их значения сожаления самые низкие.

In [73]:
data = go.Bar(
    x=[ucb1_dict['reward'], thompson_dict['reward'], epsilon_dict['reward'], random_dict['reward']],
    y=['UCB1', 'Thompson Sampling', 'e-Greedy','Random Selection'],
    orientation = 'h',
    marker=dict(color=['#33cc33', '#ff3300', '#0099ff', '#ffcc66']),
    opacity=0.7
)

text = go.Scatter(
    x=[ucb1_dict['reward'], thompson_dict['reward'], epsilon_dict['reward'], random_dict['reward']],
    y=['UCB1', 'Thompson Sampling', 'e-Greedy', 'Random Selection'],
    mode='text',
    text=[ucb1_dict['reward'], thompson_dict['reward'], epsilon_dict['reward'], random_dict['reward']],
    textposition='middle left',
    line = dict(
        color = ('rgba(255,141,41,0.6)'),
        width = 1
    ),
    textfont=dict(
        family='sans serif',
        size=16,
        color='#000000'
    )
)

data = [data,text]

layout = go.Layout(
    title='Общее вознаграждение по алгоритмам',
    xaxis={'title': 'Общая награда (клики)'},
    margin={'l':200},
    showlegend=False
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Может случиться так, что общее вознаграждение за алгоритм с наименьшим значением regret не будет самым высоким. Это связано с тем, что даже если алгоритм подберет правильное объявление, это не гарантирует, что пользователь кликнет по нему. 
Выборка Томпсона, как правило, является лучшим выбором, но также были рассмотрены другие алгоритмы. Метод который стоит выбирать зависит от каждой уникальной проблемы, имеющейся предварительной информации о кампаниях и информации, которую хотим получить впоследствии. 